## 1. Sequence becomes Image
지난시간에는 시계열 데이터에 대해서 알아보았습니다. 하지만 시계열 데이터를 반드시 Sequence의 형태로 처리하라는 법은 없습니다. 가끔은 시계열의 패턴도 이미지처럼 해석을 하는것이 더 효과적일 때가 있습니다.

**Q: 어떻게 시퀀스 데이터를 이미지로 나타낸다는 거지?**

**A: 푸리에 변환 이라고 들어보셨나요?**

### Introduction to Fourier Transform

**직관적 이해**

임의의 입력 신호를 다양한 주파수를 갖는 주기함수들의 합으로 분해하여 표현


**수학적 이해**

다음은 Discrete Fourier Transform 에 대한 수식입니다.
<img src="https://miro.medium.com/max/812/1*Hm88khvsCJE7e0kkUJ97wA.png" width=300/>

이게 도대체 뭘까?

<img src="https://pbs.twimg.com/media/DUoTATuWkAAGzhM.jpg" width=500 />

[링크 동영상 참고](https://www.youtube.com/watch?v=spUNpyF58BY)

결국 Time Domain의 데이터를 Frequency Domain으로 나타나게 해준다!

In [ ]:
# 수업자료 깃허브 Repository를 다운받아준다
!rm -rf ./NLP_2020
!git clone https://github.com/HanyangTechAI/NLP_2020.git
!rm -rf ./NLP_2020/.git

!rm -rf ./DeepIsolation
!git clone https://github.com/litcoderr/DeepIsolation.git
!rm -rf ./DeepIsolation

In [ ]:
# 필요한 패키지 (인생을 편하게 해주는 미리 만들어놓은 프로그램이라고 보면 된다) 를 설치시킨다
!cat ./NLP_2020/lab02_sequence_meets_computer_vision/requirements.txt
!echo '----------------------------------'
!pip install -r ./NLP_2020/lab02_sequence_meets_computer_vision/requirements.txt

In [ ]:
!pip install youtube_dl
!pip install pydub
!apt install ffmpeg

### 1. 유튜브에서 음성 파일을 다운로드 받아봅시다

In [ ]:
import youtube_dl

def download(link):
  ydl_opts = {
      'format': 'bestaudio/best',
      'postprocessors': [{
          'key': 'FFmpegExtractAudio',
          'preferredcodec': 'mp3',
          'preferredquality': '192',
      }],
  }
  with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    result = ydl.extract_info(link,download=False)
    if 'entries' in result:
      video = result['entries'][0]
      return 0
    else:
      video = result
      title = "{}-{}.mp3".format(video["title"],video['id'])
      ydl.download([link])
      return title

In [ ]:
link = input("유튜브 링크를 입력하시오: ")
file_path = download(link)
print('------------------------\n{} has been download'.format(file_path))

###  2. MP3 파일 -> WAV 파일 변환
MP3 파일은 음의 파형을 압축해놓은 압축 파일형식 입니다. 따라서 파형을 나타낸 WAV 형식으로 변환하는 과정이 필요합니다.

In [ ]:
import tempfile
import os
import pydub
import scipy
import scipy.io.wavfile

def read_mp3(file_path, as_float = True):
    """
    Read an MP3 File into numpy data.
    :param file_path: String path to a file
    :param as_float: Cast data to float and normalize to [-1, 1]
    :return: Tuple(rate, data), where
        rate is an integer indicating samples/s
        data is an ndarray(n_samples, 2)[int16] if as_float = False
            otherwise ndarray(n_samples, 2)[float] in range [-1, 1]
    """

    path, ext = os.path.splitext(file_path)
    assert ext=='.mp3'
    mp3 = pydub.AudioSegment.from_mp3(file_path)
    _, path = tempfile.mkstemp()
    mp3.export(path, format="wav")
    rate, data = scipy.io.wavfile.read(path)
    os.remove(path)
    if as_float:
        data = data/(2**15)
    return rate, data

In [ ]:
sample_rate, wave_data = read_mp3(file_path=file_path)

print('sample rate: {}'.format(sample_rate))
print('stereo wave vector shape: {}'.format(wave_data.shape))

# Mono 오디오 파일로 변환
wave_data = wave_data.sum(axis=1) / 2

print('mono wave vector shape: {}'.format(wave_data.shape))

위 정보를 바탕으로 간단한 이해를 하고 가봅시다.

- 샘플링 Rate 은 48000 Hz 임을 볼 수 있습니다. 즉, 1초에 48000번 파의 위치를 기록한 데이터 임을 알 수 있습니다.
- Wave 벡터의 Shape은 (num_samples, 2) 임을 알 수 있습니다. 2 인 이유는, stereo 데이터 이기 때문입니다. 즉, 왼쪽 오른쪽의 음에 대한 정보를 따로 가지고 있다고 볼 수 있습니다.

### 파형 출력

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

def plot(wave, sample_rate, start_sec, end_sec):
    start_index = int(float(start_second) * sample_rate)
    end_index = int(float(end_second) * sample_rate)
    
    plt.plot(wave[start_index:end_index])
    plt.ylabel('wave')
    plt.axis([None,None,-1,1])
    plt.show()
    
while True:
    start_second = input("시작 초: ")
    if start_second == "":
        break
    end_second = input("끝 초: ")
    
    plot(wave_data, sample_rate, float(start_second), float(end_second))

### 푸리에 변환을 해보자

In [ ]:
from DeepIsolation.modules.util.stft import STFTConverter

converter  = STFTConverter()

spectrogram = converter.get_stft(wave_data) # shape: [2(mag,angle), sample_time_step, freq]
print('spectrogram shape: {}'.format(spectrogram.shape))

In [ ]:
magnitude = spectrogram[0,:,:]
phase = spectrogram[1,:,:]

plt.imshow(magnitude, aspect='auto')
plt.xlabel("magnitude")
plt.ylabel("sample_time_step")
plt.show()

### Inverse 푸리에 변환을 통해 Frequenct 도메인 에서 Time 도메인으로 변환해보자

In [ ]:
spectrogram = np.stack((magnitude, phase))
inversed_wave_data = converter.stft2wav(spectrogram)

print(inversed_wave_data.shape)

In [ ]:
while True:
    start_second = input("시작 초: ")
    if start_second == "":
        break
    end_second = input("끝 초: ")
    
    start_index = int(float(start_second) * sample_rate)
    end_index = int(float(end_second) * sample_rate)
    plot(inversed_wave_data, sample_rate, float(start_second), float(end_second))

**이제 여러분은 시퀀스 데이터를 이미지로 처리할 준비가 되었습니다!!**